# Xente Credit Score

This challenge was hosted on <a href ="https://zindi.africa/competitions/sbtic-xente-credit-scoring-challenge">Zindi</a>. The objective of this challenge is to create a machine learning model to predict which individuals are most likely to default on their loans, based on their loan repayment behaviour and ecommerce transaction activity.

The resulting models and solutions will help Xente refine their credit decision processes, and enable them to more adequately assess the creditworthiness of new and existing clients. For Xente, this may result in improved profitability and financial sustainability; while for Xente’s cliente, increased creditworthiness would enhance their access to credit and contribute to an improved livelihood.

In [1]:
# Data Wrangling 
import numpy as np
import pandas as pd

## Model evaluators
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report,accuracy_score

## Models to be used
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier

# Remove unnecessary warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
V_Desc = pd.read_csv('VariableDefinitions.csv')
V_Desc.head(27)

,Variable,Definition
0,CustomerId,Unique number identifying the customer on plat...
1,TransactionStartTime,Transaction start time
2,Value,Value of transaction
3,Amount,Value of Transaction with charges
4,TransactionId,Unique transaction identifier on platform
5,BatchId,Identifier for bulk transactions being done on...
6,SubscriptionId,You can have one account with multiple subscri...
7,CurrencyCode,Country currency
8,CountryCode,Numerical geographical code of country
9,ProviderId,Source provider of Item bought


In [3]:
df = pd.read_csv('Train.csv')
df.head(10)

,CustomerId,TransactionStartTime,Value,Amount,TransactionId,BatchId,SubscriptionId,CurrencyCode,CountryCode,ProviderId,...,LoanId,PaidOnDate,IsFinalPayBack,InvestorId,DueDate,LoanApplicationId,PayBackId,ThirdPartyId,IsThirdPartyConfirmed,IsDefaulted
0,CustomerId_27,2018-09-21 12:17:39,550.0,-550.0,TransactionId_1683,BatchId_641,SubscriptionId_2,UGX,256,ProviderId_1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,CustomerId_27,2018-09-25 09:20:29,550.0,-550.0,TransactionId_2235,BatchId_820,SubscriptionId_2,UGX,256,ProviderId_1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,CustomerId_27,2018-09-25 10:33:31,550.0,-550.0,TransactionId_1053,BatchId_210,SubscriptionId_4,UGX,256,ProviderId_1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,CustomerId_27,2018-09-27 10:26:41,1000.0,-1000.0,TransactionId_2633,BatchId_876,SubscriptionId_4,UGX,256,ProviderId_1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,CustomerId_27,2018-09-27 12:44:21,500.0,-500.0,TransactionId_71,BatchId_1362,SubscriptionId_4,UGX,256,ProviderId_1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,CustomerId_27,2018-09-28 12:40:48,500.0,-500.0,TransactionId_930,BatchId_357,SubscriptionId_4,UGX,256,ProviderId_1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,CustomerId_27,2018-10-03 09:34:04,500.0,-500.0,TransactionId_1381,BatchId_8,SubscriptionId_4,UGX,256,ProviderId_1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,CustomerId_27,2018-10-17 09:06:16,1235.0,-1235.0,TransactionId_255,BatchId_1082,SubscriptionId_4,UGX,256,ProviderId_1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,CustomerId_27,2018-10-17 12:02:45,1235.0,-1235.0,TransactionId_543,BatchId_1195,SubscriptionId_4,UGX,256,ProviderId_1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,CustomerId_258,2018-10-18 16:11:55,10000.0,-10000.0,TransactionId_1041,BatchId_1970,SubscriptionId_4,UGX,256,ProviderId_1,...,LoanId_62,2018-10-22 09:13:17,1.0,InvestorId_3,2018-11-17 16:11:04,NaN,PayBackId_1791,ThirdPartyId_648,0.0,0.0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2100 entries, 0 to 2099
Data columns (total 27 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   CustomerId             2100 non-null   object 
 1   TransactionStartTime   2100 non-null   object 
 2   Value                  2100 non-null   float64
 3   Amount                 2100 non-null   float64
 4   TransactionId          2100 non-null   object 
 5   BatchId                2100 non-null   object 
 6   SubscriptionId         2100 non-null   object 
 7   CurrencyCode           2100 non-null   object 
 8   CountryCode            2100 non-null   int64  
 9   ProviderId             2100 non-null   object 
 10  ProductId              2100 non-null   object 
 11  ProductCategory        2100 non-null   object 
 12  ChannelId              2100 non-null   object 
 13  TransactionStatus      2100 non-null   int64  
 14  IssuedDateLoan         1488 non-null   object 
 15  Amou

In [5]:
df['IsDefaulted'].value_counts()

0.0    1310
1.0     178
Name: IsDefaulted, dtype: int64

In [6]:
test = pd.read_csv('Test.csv')
test.head(10)

,CustomerId,TransactionStartTime,Value,Amount,TransactionId,BatchId,SubscriptionId,CurrencyCode,CountryCode,ProviderId,ProductId,ProductCategory,ChannelId,TransactionStatus,IssuedDateLoan,LoanId,InvestorId,LoanApplicationId,ThirdPartyId
0,CustomerId_310,2019-03-31 13:33:05,14000.0,-14000.0,TransactionId_925,BatchId_1144,SubscriptionId_7,UGX,256,ProviderId_1,ProductId_7,airtime,ChannelId_1,1,2019-03-31 13:33:04,LoanId_1027,InvestorId_1,LoanApplicationId_825,ThirdPartyId_1175
1,CustomerId_243,2019-03-31 15:04:09,1000.0,-1000.0,TransactionId_1080,BatchId_1214,SubscriptionId_7,UGX,256,ProviderId_1,ProductId_8,data_bundles,ChannelId_1,1,2019-03-31 15:04:08,LoanId_768,InvestorId_1,LoanApplicationId_68,ThirdPartyId_604
2,CustomerId_142,2019-03-31 17:31:11,2500.0,-2500.0,TransactionId_2315,BatchId_2150,SubscriptionId_7,UGX,256,ProviderId_1,ProductId_7,airtime,ChannelId_1,1,2019-03-31 17:31:09,LoanId_1067,InvestorId_1,LoanApplicationId_1223,ThirdPartyId_1521
3,CustomerId_142,2019-03-31 17:32:15,500.0,-500.0,TransactionId_1466,BatchId_1071,SubscriptionId_7,UGX,256,ProviderId_1,ProductId_7,airtime,ChannelId_1,1,2019-03-31 17:32:14,LoanId_202,InvestorId_1,LoanApplicationId_633,ThirdPartyId_406
4,CustomerId_142,2019-03-31 17:34:41,1000.0,-1000.0,TransactionId_337,BatchId_2477,SubscriptionId_7,UGX,256,ProviderId_1,ProductId_7,airtime,ChannelId_1,1,2019-03-31 17:34:40,LoanId_533,InvestorId_1,LoanApplicationId_309,ThirdPartyId_302
5,CustomerId_107,2019-03-31 17:40:20,1500.0,-1500.0,TransactionId_2098,BatchId_1923,SubscriptionId_7,UGX,256,ProviderId_1,ProductId_3,airtime,ChannelId_1,0,NaN,NaN,NaN,NaN,NaN
6,CustomerId_107,2019-03-31 17:40:26,1500.0,-1500.0,TransactionId_608,BatchId_1923,SubscriptionId_7,UGX,256,ProviderId_1,ProductId_3,airtime,ChannelId_1,0,NaN,NaN,NaN,NaN,NaN
7,CustomerId_142,2019-03-31 17:44:48,500.0,-500.0,TransactionId_875,BatchId_1558,SubscriptionId_7,UGX,256,ProviderId_1,ProductId_7,airtime,ChannelId_1,1,2019-03-31 17:44:47,LoanId_1469,InvestorId_1,LoanApplicationId_247,ThirdPartyId_281
8,CustomerId_407,2019-03-31 22:23:09,4978.0,-4250.0,TransactionId_2604,BatchId_683,SubscriptionId_7,UGX,256,ProviderId_1,ProductId_15,utility_bill,ChannelId_1,0,NaN,NaN,NaN,NaN,NaN
9,CustomerId_407,2019-03-31 22:40:34,4720.0,-4000.0,TransactionId_1565,BatchId_1876,SubscriptionId_7,UGX,256,ProviderId_1,ProductId_15,utility_bill,ChannelId_1,0,NaN,NaN,NaN,NaN,NaN


In [7]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 905 entries, 0 to 904
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   CustomerId            905 non-null    object 
 1   TransactionStartTime  905 non-null    object 
 2   Value                 905 non-null    float64
 3   Amount                905 non-null    float64
 4   TransactionId         905 non-null    object 
 5   BatchId               905 non-null    object 
 6   SubscriptionId        905 non-null    object 
 7   CurrencyCode          905 non-null    object 
 8   CountryCode           905 non-null    int64  
 9   ProviderId            905 non-null    object 
 10  ProductId             905 non-null    object 
 11  ProductCategory       905 non-null    object 
 12  ChannelId             905 non-null    object 
 13  TransactionStatus     905 non-null    int64  
 14  IssuedDateLoan        478 non-null    object 
 15  LoanId                4

In [8]:
df.isnull().sum()

CustomerId                 0
TransactionStartTime       0
Value                      0
Amount                     0
TransactionId              0
BatchId                    0
SubscriptionId             0
CurrencyCode               0
CountryCode                0
ProviderId                 0
ProductId                  0
ProductCategory            0
ChannelId                  0
TransactionStatus          0
IssuedDateLoan           612
AmountLoan               612
Currency                 612
LoanId                   612
PaidOnDate               612
IsFinalPayBack           612
InvestorId               612
DueDate                  614
LoanApplicationId        617
PayBackId                612
ThirdPartyId             614
IsThirdPartyConfirmed    612
IsDefaulted              612
dtype: int64

In [9]:
test.isnull().sum()

CustomerId                0
TransactionStartTime      0
Value                     0
Amount                    0
TransactionId             0
BatchId                   0
SubscriptionId            0
CurrencyCode              0
CountryCode               0
ProviderId                0
ProductId                 0
ProductCategory           0
ChannelId                 0
TransactionStatus         0
IssuedDateLoan          427
LoanId                  427
InvestorId              427
LoanApplicationId       427
ThirdPartyId            427
dtype: int64

In [10]:
df.shape

(2100, 27)

In [11]:
test.shape

(905, 19)

In [12]:
#Changing travel_date data type to date
df["TransactionStartTime"] = pd.to_datetime(df["TransactionStartTime"], infer_datetime_format = True)

#Changing travel_data to year, month and day of the week
df["days of week"] = df["TransactionStartTime"].dt.dayofweek
df["year"] = df["TransactionStartTime"].dt.year
df["month"] = df["TransactionStartTime"].dt.month

In [13]:
#Changing travel_date data type to date
test["TransactionStartTime"] = pd.to_datetime(test["TransactionStartTime"], infer_datetime_format = True)

#Changing travel_data to year, month and day of the week
test["days of week"] = test["TransactionStartTime"].dt.dayofweek
test["year"] = test["TransactionStartTime"].dt.year
test["month"] = test["TransactionStartTime"].dt.month

In [14]:
df.drop(['CustomerId', 'TransactionStartTime', 'CurrencyCode', 'CountryCode',  'BatchId', 'IssuedDateLoan', 'PayBackId', 'SubscriptionId', 'IsThirdPartyConfirmed',
           'AmountLoan', 'Currency', 'LoanId','PaidOnDate', 'IsFinalPayBack','DueDate','ProviderId', 'LoanApplicationId', 'ThirdPartyId', 'ChannelId'], axis = 1, inplace = True)
test.drop(['CustomerId', 'TransactionStartTime', 'CurrencyCode', 'CountryCode', 'BatchId', 'IssuedDateLoan', 'LoanId', 'SubscriptionId', 'LoanApplicationId', 'ThirdPartyId', 'ProviderId', 'ChannelId'], axis = 1, inplace = True)

In [21]:
df.head()

,Value,Amount,TransactionId,TransactionStatus,IsDefaulted,days of week,year,month,ProductCategory_airtime,ProductCategory_data_bundles,...,ProductId_ProductId_3,ProductId_ProductId_4,ProductId_ProductId_5,ProductId_ProductId_6,ProductId_ProductId_7,ProductId_ProductId_8,ProductId_ProductId_9,InvestorId_InvestorId_1,InvestorId_InvestorId_2,InvestorId_InvestorId_3
9,10000.0,-10000.0,TransactionId_1041,1,0.0,3,2018,10,1,0,...,0,0,0,0,1,0,0,0,0,1
10,10000.0,-10000.0,TransactionId_1041,1,0.0,3,2018,10,1,0,...,0,0,0,0,1,0,0,0,0,1
11,10000.0,-10000.0,TransactionId_1041,1,0.0,3,2018,10,1,0,...,0,0,0,0,1,0,0,0,0,1
13,5150.0,-5150.0,TransactionId_1598,1,0.0,4,2018,10,1,0,...,0,0,0,0,1,0,0,0,0,1
14,515.0,-515.0,TransactionId_2115,1,0.0,4,2018,10,1,0,...,0,0,0,0,1,0,0,0,0,1


In [22]:
test.head()

,Value,Amount,TransactionId,TransactionStatus,days of week,year,month,ProductCategory_airtime,ProductCategory_data_bundles,ProductCategory_financial_services,...,ProductId_ProductId_19,ProductId_ProductId_3,ProductId_ProductId_4,ProductId_ProductId_5,ProductId_ProductId_6,ProductId_ProductId_7,ProductId_ProductId_8,ProductId_ProductId_9,InvestorId_InvestorId_1,InvestorId_InvestorId_2
0,14000.0,-14000.0,TransactionId_925,1,6,2019,3,1,0,0,...,0,0,0,0,0,1,0,0,1,0
1,1000.0,-1000.0,TransactionId_1080,1,6,2019,3,0,1,0,...,0,0,0,0,0,0,1,0,1,0
2,2500.0,-2500.0,TransactionId_2315,1,6,2019,3,1,0,0,...,0,0,0,0,0,1,0,0,1,0
3,500.0,-500.0,TransactionId_1466,1,6,2019,3,1,0,0,...,0,0,0,0,0,1,0,0,1,0
4,1000.0,-1000.0,TransactionId_337,1,6,2019,3,1,0,0,...,0,0,0,0,0,1,0,0,1,0


In [17]:
df.dropna(axis = 0, inplace = True)
test.dropna(axis = 0, inplace = True)

In [18]:
df['InvestorId'].value_counts()

InvestorId_1    1068
InvestorId_2     414
InvestorId_3       6
Name: InvestorId, dtype: int64

In [20]:
df = pd.get_dummies(df, columns = ['ProductCategory', 'ProductId', 'InvestorId'])
test = pd.get_dummies(test, columns = ['ProductCategory', 'ProductId', 'InvestorId'])

In [24]:
x = df.drop(['TransactionId', 'IsDefaulted'], axis = 1)
y = df['IsDefaulted']

In [25]:
x_train, x_test,y_train, y_test = train_test_split(x, y, test_size = 0.20, random_state = 42)

In [26]:
rf = RandomForestClassifier()
rf.fit(x_train,y_train)
rf_pred = rf.predict(x_test)

In [27]:
rf.score(x_test, y_test)

0.9228187919463087

In [29]:
cm = confusion_matrix(y_test, rf_pred)
print(cm)
print('\n')
print(classification_report(y_test, rf_pred))

[[250  12]
 [ 11  25]]


              precision    recall  f1-score   support

         0.0       0.96      0.95      0.96       262
         1.0       0.68      0.69      0.68        36

    accuracy                           0.92       298
   macro avg       0.82      0.82      0.82       298
weighted avg       0.92      0.92      0.92       298



In [30]:
print (f'Mean Absolute Error =', mean_absolute_error(rf_pred, y_test))
rmse = mean_squared_error(y_test, rf_pred, squared=False)
print(f"Root Mean Squared Error = {rmse / 1e6:.3} Mbit/s")

Mean Absolute Error = 0.07718120805369127
Root Mean Squared Error = 2.78e-07 Mbit/s


In [31]:
xgb = XGBClassifier(
    n_jobs=-1,
    random_state=101,
    max_depth=5,
    n_estimators=400,
)
xgb.fit(x_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=5, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=400,
              n_jobs=-1, num_parallel_tree=1, predictor='auto',
              random_state=101, reg_alpha=0, reg_lambda=1, ...)

In [32]:
predict = xgb.predict(x_test)
xgb.score(x_test, y_test)

0.912751677852349

In [33]:
cm = confusion_matrix(y_test, predict)
print(cm)
print('\n')
print(classification_report(y_test, predict))

[[242  20]
 [  6  30]]


              precision    recall  f1-score   support

         0.0       0.98      0.92      0.95       262
         1.0       0.60      0.83      0.70        36

    accuracy                           0.91       298
   macro avg       0.79      0.88      0.82       298
weighted avg       0.93      0.91      0.92       298



In [34]:
print (f'Mean Absolute Error =', mean_absolute_error(predict, y_test))
rmse = mean_squared_error(y_test, predict, squared=False)
print(f"Root Mean Squared Error = {rmse / 1e6:.3} Mbit/s")

Mean Absolute Error = 0.087248322147651
Root Mean Squared Error = 2.95e-07 Mbit/s


In [37]:
from sklearn.model_selection import RandomizedSearchCV

# define the hyperparameter space
param_dist = {
    "n_estimators": range(100, 600, 100),
    "max_depth": range(3, 10),
    "learning_rate": [0.1, 0.01, 0.05],
    "subsample": [0.5, 0.6, 0.7, 0.8, 0.9],
    "colsample_bytree": [0.5, 0.6, 0.7, 0.8, 0.9],
    "min_child_weight": range(1, 5)
}

# initialize the xgboost classifier
random_xgb = XGBClassifier(objective="binary:logistic")

# initialize the random search
random_search = RandomizedSearchCV(
    estimator=random_xgb,
    param_distributions=param_dist,
    n_iter=50,
    scoring="accuracy",
    cv=5,
    verbose=1,
    n_jobs=-1,
    random_state=42
)

# fit the random search on the data
random_search.fit(x_train, y_train)

# print the best parameters and score
print("Best parameters:", random_search.best_params_)
print("Best score:", random_search.best_score_)

Fitting 5 folds for each of 50 candidates, totalling 250 fits
Best parameters: {'subsample': 0.5, 'n_estimators': 300, 'min_child_weight': 3, 'max_depth': 4, 'learning_rate': 0.01, 'colsample_bytree': 0.5}
Best score: 0.9394957983193277


In [38]:
# define the hyperparameter space
param_dist = {
    "n_estimators": range(100, 600, 100),
    "max_depth": range(3, 10),
    "learning_rate": [0.1, 0.01, 0.05],
    "subsample": [0.5, 0.6, 0.7, 0.8, 0.9],
    "colsample_bytree": [0.5, 0.6, 0.7, 0.8, 0.9],
    "min_child_weight": range(1, 5)
}

# initialize the xgboost classifier
random = XGBClassifier()

# initialize the random search
randomXgb_search = RandomizedSearchCV(
    estimator=random,
    param_distributions=param_dist,
    n_iter=50,
    scoring="accuracy",
    cv=5,
    verbose=1,
    n_jobs=-1,
    random_state=42
)

# fit the random search on the data
randomXgb_search.fit(x_train, y_train)

# print the best parameters and score
print("Best parameters:", randomXgb_search.best_params_)
print("Best score:", randomXgb_search.best_score_)

Fitting 5 folds for each of 50 candidates, totalling 250 fits
Best parameters: {'subsample': 0.5, 'n_estimators': 300, 'min_child_weight': 3, 'max_depth': 4, 'learning_rate': 0.01, 'colsample_bytree': 0.5}
Best score: 0.9394957983193277


In [40]:
from sklearn.model_selection import GridSearchCV

# define the hyperparameter space
param_grid = {
    "n_estimators": [100, 200, 300, 400, 500],
    "max_depth": [3, 4, 5, 6, 7, 8, 9],
    "learning_rate": [0.1, 0.01, 0.05],
    "subsample": [0.5, 0.6, 0.7, 0.8, 0.9],
    "colsample_bytree": [0.5, 0.6, 0.7, 0.8, 0.9],
    "min_child_weight": [1, 2, 3, 4]
}

# initialize the xgboost classifier
grid_xgb = XGBClassifier()

# initialize the grid search
grid_search = GridSearchCV(
    estimator=grid_xgb,
    param_grid=param_grid
    
    scoring="accuracy",
    cv=5,
    verbose=1,
    n_jobs=-1
)

# fit the grid search on the data
grid_search.fit(x_train, y_train)

# print the best parameters and score
print("Best parameters:", grid_search.best_params_)
print("Best score:", grid_search.best_score_)

Fitting 5 folds for each of 10500 candidates, totalling 52500 fits
Best parameters: {'colsample_bytree': 0.5, 'learning_rate': 0.01, 'max_depth': 3, 'min_child_weight': 1, 'n_estimators': 100, 'subsample': 0.6}
Best score: 0.9403361344537815
